In [4]:
from pymongo import MongoClient
import pandas as pd
import numpy as np

%matplotlib

Using matplotlib backend: MacOSX


In [6]:
client=MongoClient("mongodb://localhost:27017/")
db=client.companies

In [9]:
#Query para pedir toda la base de datos

companies=db.companies.find()


In [10]:
datos= pd.DataFrame(companies)
datos.head()

In [11]:
datos.head()

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,video_embeds,screenshots,external_links,partners,founded_month,founded_day,deadpooled_month,deadpooled_day,deadpooled_url,ipo
0,52cdef7c4bab8bd675297d8b,AdventNet,abc3,http://www.crunchbase.com/company/adventnet,http://adventnet.com,,,manageengine,enterprise,600.0,...,[],"[{'available_sizes': [[[150, 94], 'assets/imag...",[],[],NaN,NaN,NaN,NaN,NaN,NaN
1,52cdef7c4bab8bd675297d8f,Omnidrive,omnidrive,http://www.crunchbase.com/company/omnidrive,http://www.omnidrive.com,http://www.omnidrive.com/blog,http://feeds.feedburner.com/omnidrive,Nomadesk,network_hosting,NaN,...,[],[],[],[],11.0,1.0,9.0,15.0,,None
2,52cdef7c4bab8bd675297d92,Flektor,flektor,http://www.crunchbase.com/company/flektor,http://www.flektor.com,http://www.flektor-blog.com,http://www.flektor-blog.com/video_editing_soft...,None,games_video,NaN,...,[],[],[],[],NaN,NaN,NaN,NaN,None,None
3,52cdef7c4bab8bd675297d90,Postini,postini,http://www.crunchbase.com/company/postini,http://postini.com,,,None,web,NaN,...,[],[],[],[],6.0,2.0,NaN,NaN,None,None
4,52cdef7c4bab8bd675297d91,Geni,geni,http://www.crunchbase.com/company/geni,http://www.geni.com,http://blog.geni.com,http://blog.geni.com/index.rdf,geni,web,18.0,...,"[{'embed_code': '<object width=""425"" height=""3...",[],[],[],6.0,1.0,NaN,NaN,None,None
